In [259]:
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine
import os, sys
import json
import pandas as pd



In [260]:
from src.helper import Helper
helper = Helper()

In [261]:
# Connection parameters
connection_params = {
    "host": "localhost",
    "user": "birehan",
    "password": "password",
    "port": "5432",
    "database": "slackdbs"  # Change to your desired database name
}

# Create a connection to PostgreSQL
conn = psycopg2.connect(**connection_params)
conn.autocommit = True  # Enable autocommit to execute DDL statements

# Create tables for Slack messages, replies, reactions, users, and channels
with conn.cursor() as cursor:
    # Users table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Users (
        user_id VARCHAR(255) PRIMARY KEY,
        real_name VARCHAR(255) NOT NULL
    )
    """)

    # Channels table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Channels (
        channel_id VARCHAR(255) PRIMARY KEY,
        name VARCHAR(255) NOT NULL
    )
    """)

    # Messages table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Messages (
        ts VARCHAR(255) PRIMARY KEY,
        msg_id VARCHAR(255) NOT NULL,
        text TEXT NOT NULL,
        cleaned_text TEXT,
        user_id VARCHAR(255) REFERENCES Users(user_id),
        channel_id VARCHAR(255) REFERENCES Channels(channel_id),
        mentions VARCHAR(255)[],
        reactions INT[],
        replies INT[]
    )
    """)

    # Reactions table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Reactions (
        reaction_id SERIAL PRIMARY KEY,
        message_ts VARCHAR(255) REFERENCES Messages(ts),
        name VARCHAR(255) NOT NULL,
        count INT NOT NULL
    )
    """)

    # Replies table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Replies (
        reply_id SERIAL PRIMARY KEY,
        message_ts VARCHAR(255) REFERENCES Messages(ts),
        user_id VARCHAR(255) REFERENCES Users(user_id),
        ts VARCHAR(255) NOT NULL
    )
    """)

    # Bridge table for Messages and Reactions (many-to-many)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS MessageReactions (
        message_ts VARCHAR(255) REFERENCES Messages(ts),
        reaction_id INT REFERENCES Reactions(reaction_id),
        PRIMARY KEY (message_ts, reaction_id)
    )
    """)

    # Bridge table for Messages and Replies (many-to-many)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS MessageReplies (
        message_ts VARCHAR(255) REFERENCES Messages(ts),
        reply_id INT REFERENCES Replies(reply_id),
        PRIMARY KEY (message_ts, reply_id)
    )
    """)

    # Bridge table for Messages and Mentions (many-to-many)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS MessageMentions (
        message_ts VARCHAR(255) REFERENCES Messages(ts),
        mentioned_user_id VARCHAR(255) REFERENCES Users(user_id),
        PRIMARY KEY (message_ts, mentioned_user_id)
    )
    """)

    # Bridge table for Reactions and Users (many-to-many)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS ReactionUsers (
        reaction_id INT REFERENCES Reactions(reaction_id),
        user_id VARCHAR(255) REFERENCES Users(user_id),
        PRIMARY KEY (reaction_id, user_id)
    )
    """)

# Close the connection
conn.close()

# Check that tables were created
engine = create_engine(f"postgresql+psycopg2://{connection_params['user']}:{connection_params['password']}@{connection_params['host']}:{connection_params['port']}/{connection_params['database']}")


## Insert all users into the postgress database

In [262]:

all_users = helper.get_all_users()
print("total users: ", len(all_users))
print(all_users[:10])

total users:  71
[{'user_id': 'U03T89ACUUW', 'real_name': 'Carlos Gross'}, {'user_id': 'U03TEPYRM2P', 'real_name': 'Garrett Bell'}, {'user_id': 'U03TNP8Q8CT', 'real_name': 'Bethany George'}, {'user_id': 'U03TT5KEYCF', 'real_name': 'Luis Ingram'}, {'user_id': 'U03TX2VN6H5', 'real_name': 'Nicole Kim'}, {'user_id': 'U03U1FNPEUX', 'real_name': 'Joshua Rhodes'}, {'user_id': 'U03U1FQKEMV', 'real_name': 'Steven Garcia'}, {'user_id': 'U03U1GHT39V', 'real_name': 'Joseph Diaz'}, {'user_id': 'U03U1HAG9TR', 'real_name': 'Robert Carter'}, {'user_id': 'U03U1J51VFZ', 'real_name': 'Cheryl Hudson'}]


In [263]:
conn = psycopg2.connect(**connection_params)
conn.autocommit = True  # Enable autocommit to execute DDL statements

with conn.cursor() as cursor:
    for user in all_users:
        cursor.execute("""
        INSERT INTO Users (user_id, real_name)
        VALUES (%s, %s)
        ON CONFLICT (user_id) DO NOTHING
        """, (user['user_id'], user['real_name']))

# Close the connection
conn.close()

In [264]:
# Create a connection to PostgreSQL
conn = psycopg2.connect(**connection_params)

# Create a cursor
cursor = conn.cursor()

# Get the number of rows in the Users table
cursor.execute("SELECT COUNT(*) FROM Users;")
num_rows = cursor.fetchone()[0]
print(f"Number of rows in Users table: {num_rows}")

# Get the first 5 persons from the Users table
cursor.execute("SELECT * FROM Users LIMIT 5;")
first_five_persons = cursor.fetchall()
print("First 5 persons:")
for person in first_five_persons:
    print(person)

# Close the cursor and connection
cursor.close()
conn.close()

Number of rows in Users table: 71
First 5 persons:
('U03T89ACUUW', 'Carlos Gross')
('U03TEPYRM2P', 'Garrett Bell')
('U03TNP8Q8CT', 'Bethany George')
('U03TT5KEYCF', 'Luis Ingram')
('U03TX2VN6H5', 'Nicole Kim')


## Insert all channels into the postgress database

In [265]:
all_channels = helper.get_all_channels()
    
print("total channels: ", len(all_channels))
print(all_channels[:10])

total channels:  39
[{'channel_id': 'C03T0APHX63', 'name': 'all-community-building'}, {'channel_id': 'C03T0AX4K6K', 'name': 'all-technical-support'}, {'channel_id': 'C03T89KDGA2', 'name': 'all-career-exercises'}, {'channel_id': 'C03T89PMJKG', 'name': 'all-resources'}, {'channel_id': 'C03TBUCU4UD', 'name': 'random'}, {'channel_id': 'C03TEQM38HH', 'name': 'all-ideas'}, {'channel_id': 'C03TEQQS9NF', 'name': 'all-week1'}, {'channel_id': 'C03U4J8J4LQ', 'name': 'all-broadcast'}, {'channel_id': 'C03UG4LHM8A', 'name': 'tenx-bot'}, {'channel_id': 'C03V3LK61QX', 'name': 'team-10'}]


In [266]:
# Create a connection to PostgreSQL
conn = psycopg2.connect(**connection_params)

# Create a cursor
cursor = conn.cursor()

# Insert channels into the Channels table
for channel in all_channels:
    cursor.execute(
        "INSERT INTO Channels (channel_id, name) VALUES (%s, %s) ON CONFLICT (channel_id) DO NOTHING;",
        (channel['channel_id'], channel['name'])
    )

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [267]:
# Create a connection to PostgreSQL
conn = psycopg2.connect(**connection_params)

# Create a cursor
cursor = conn.cursor()

# Get the total number of channels
cursor.execute("SELECT COUNT(*) FROM Channels;")
total_channels = cursor.fetchone()[0]
print(f"Total number of channels: {total_channels}")

# Print the top 5 channels
cursor.execute("SELECT * FROM Channels LIMIT 5;")
top_channels = cursor.fetchall()
print("\nTop 5 channels:")
for channel in top_channels:
    print(f"Channel ID: {channel[0]}, Name: {channel[1]}")

# Close the cursor and connection
cursor.close()
conn.close()

Total number of channels: 39

Top 5 channels:
Channel ID: C03T0APHX63, Name: all-community-building
Channel ID: C03T0AX4K6K, Name: all-technical-support
Channel ID: C03T89KDGA2, Name: all-career-exercises
Channel ID: C03T89PMJKG, Name: all-resources
Channel ID: C03TBUCU4UD, Name: random


## Insert all messages into the postgress database

In [276]:
all_messages = helper.get_all_messages()
    
print("total messages: ", len(all_messages))
print(all_messages[:5])

total messages:  18944
[{'msg_id': '16f68d4e-0ceb-448a-b660-d5ef2eb05305', 'text': '*HOTSEAT ANNOUNCEMENT*', 'cleaned_text': 'hotseat announc', 'user_id': 'U03V1AM5TFA', 'mentions': [], 'reactions': [], 'replies': [], 'ts': '1662620680.298449', 'channel_id': 'C03T0APHX63'}, {'msg_id': '7c641275-2e52-4074-9894-744f049d5377', 'text': "*<!here>* Good morning Community! We are very happy and excited to announce that today's hot seat will be hosting our very own <@U03U1GHT39V>  :wink::clap::skin-tone-2::confetti_ball:   Let's prep our burning questions for him!", 'cleaned_text': 'good morn commun happi excit announc today hot seat host winkclapskintoneconfettibal let prep burn question', 'user_id': 'U03V1AM5TFA', 'mentions': ['U03U1GHT39V'], 'reactions': [{'name': 'fire', 'users': ['U03U9FWPNCE'], 'count': 1}], 'replies': [], 'ts': '1662620806.359419', 'channel_id': 'C03T0APHX63'}, {'msg_id': '245ecc4d-2c1b-4bee-b280-a1fd5ab7fee3', 'text': '*<!here> Community Building Session REMINDER!*:tim

In [279]:
# Create a connection to PostgreSQL
conn = psycopg2.connect(**connection_params)
conn.autocommit = True

# Create a cursor
cursor = conn.cursor()

# Insert messages and related data
for message in unique_json_list:
    # Check if user_id exists in Users table
    cursor.execute("""
        SELECT user_id FROM Users WHERE user_id = %s
    """, (message['user_id'],))

    user_exists = cursor.fetchone()

    if user_exists:
        # Insert into Messages table
        cursor.execute("""
            INSERT INTO Messages (ts, msg_id, text, user_id, channel_id)
            VALUES (%s, %s, %s, %s, %s) ON CONFLICT (ts) DO NOTHING
        """, (
            message['ts'], message['msg_id'], message['text'], message['user_id'],
            message['channel_id']
        ))

        # Insert into MessageMentions table
        for mention in message.get('mentions', []):
            cursor.execute("""
                INSERT INTO MessageMentions (message_ts, mentioned_user_id)
                VALUES (%s, %s) ON CONFLICT (message_ts, mentioned_user_id) DO NOTHING
            """, (message['ts'], mention))

        # Insert into Reactions table
        for reaction in message.get('reactions', []):
            cursor.execute("""
                INSERT INTO Reactions (message_ts, name, count)
                VALUES (%s, %s, %s) 
            """, (message['ts'], reaction['name'], reaction['count']))

            # Check if the reaction was inserted or already exists
            cursor.execute("""
                SELECT reaction_id FROM Reactions WHERE message_ts = %s AND name = %s
            """, (message['ts'], reaction['name']))

            result = cursor.fetchone()
            if result:
                reaction_id = result[0]

                # Insert into ReactionUsers table
                for user in reaction.get('users', []):
                    cursor.execute("""
                        INSERT INTO ReactionUsers (reaction_id, user_id)
                        VALUES (%s, %s) ON CONFLICT (reaction_id, user_id) DO NOTHING
                    """, (reaction_id, user))

        # Insert into Replies table
        for reply in message.get('replies', []):
            cursor.execute("""
                INSERT INTO Replies (message_ts, user_id, ts)
                VALUES (%s, %s, %s)
            """, (message['ts'], reply['user_id'], reply['ts']))

# Close the cursor and connection
cursor.close()
conn.close()

In [280]:
# Create a connection to PostgreSQL
conn = psycopg2.connect(**connection_params)
conn.autocommit = True

# Create a cursor
cursor = conn.cursor()

# Count the number of messages
cursor.execute("SELECT COUNT(*) FROM Messages")
message_count = cursor.fetchone()[0]
print(f"Total number of messages: {message_count}")

# Retrieve the first 5 messages
cursor.execute("SELECT * FROM Messages LIMIT 5")
first_5_messages = cursor.fetchall()

# Display the first 5 messages
print("\nFirst 5 messages:")
for message in first_5_messages:
    print(message)

# Close the cursor and connection
cursor.close()
conn.close()

Total number of messages: 18938

First 5 messages:
('1662620680.298449', '16f68d4e-0ceb-448a-b660-d5ef2eb05305', '*HOTSEAT ANNOUNCEMENT*', 'U03V1AM5TFA', 'C03T0APHX63', None, None, None)
('1662620806.359419', '7c641275-2e52-4074-9894-744f049d5377', "*<!here>* Good morning Community! We are very happy and excited to announce that today's hot seat will be hosting our very own <@U03U1GHT39V>  :wink::clap::skin-tone-2::confetti_ball:   Let's prep our burning questions for him!", 'U03V1AM5TFA', 'C03T0APHX63', None, None, None)
('1662637801.047169', '245ecc4d-2c1b-4bee-b280-a1fd5ab7fee3', '*<!here> Community Building Session REMINDER!*:timer_clock:\n*Please note that CBS is on in the next 10min*\n• *Session:* Scavenger hunt\n• *Time*: From 12:00 PM - 12:30 PM UTC\n• *Platform: Gmeet &amp; Slack*\n', 'U03V1AM5TFA', 'C03T0APHX63', None, None, None)
('1662638240.075389', 'fe80aff2-20f2-42ad-94a8-8b48ac63083f', 'Sweet music on Google meet now\n:point_right: <http://meet.google.com/ysb-kjdn-hpp|m